In [9]:
import os
import json
import re
import nltk
import spacy
import PyPDF2

from openai import OpenAI

In [2]:
os.environ['OPENAI_API_KEY'] = ''

In [10]:
# function that extracts text from a PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        print("Number of PDF pages:", len(reader.pages))
        text = ""
        for  page in reader.pages:
            page_text = page.extract_text()
            text += page_text
            #print(page_text)
    return text

# Chunking

In [3]:
def split_sentences(text):
    
    sentences = re.split('[.|?]', text)
    sentences = [sentence.strip() for sentence in sentences if sentence.split()]

    return sentences

def split_sentences_by_nltk(text):

    chunks = []
    for sentence in nltk.sent_tokenize(text):
        chunks.append(sentence)
    
    return chunks

def split_sentences_by_spacy(text, max_tokens, overlap=0, model='en_core_web_sm'):

    nlp = spacy.load(model)
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]

    return sentences

    tokens_lengths = [count_tokens(sent) for sent in sentences]

    chunks = []
    start_idx = 0

    while start_idx < len(sentences):

        current_chunk = []
        current_token_count = 0
        for idx in range(start_idx, len(sentences)):
            if current_token_count + tokens_lengths[idx] > max_tokens:
                break

            current_chunk.append(sentences[idx])
            current_token_count += tokens_lengths[idx]

        chunks.append(' '.join(current_chunk))

        if overlap >= len(current_chunk):
            start_idx += 1
        else:
            start_idx += len(current_chunk) - overlap


    return chunks

In [4]:
texto = """Entry requirements for Spain are as follows:
              - Passport with validity of at least six months.
              - Return ticket.
           Entry requirements for China are as follows:
              - Passport with validity of at least six months.
              - Return ticket.
              - Visa.
              - Proof of accommodation."""

In [6]:
s_textos = split_sentences_by_nltk(texto)

for i in range(len(s_textos)):
    print(i, s_textos[i])

0 Entry requirements for Spain are as follows:
              - Passport with validity of at least six months.
1 - Return ticket.
2 Entry requirements for China are as follows:
              - Passport with validity of at least six months.
3 - Return ticket.
4 - Visa.
5 - Proof of accommodation.


In [20]:
s_textos = split_sentences_by_spacy(texto, max_tokens=20)
for i in range(0, len(s_textos)):
   print(i, s_textos[i])

0 Entry requirements for Spain are as follows:
              - Passport with validity of at least six months.
              - Return ticket.
           
1 Entry requirements for China are as follows:
              - Passport with validity of at least six months.
              - Return ticket.
              - Visa.
              
2 - Proof of accommodation.


# PDF

In [13]:
texto_pdf = extract_text_from_pdf('/media/armando/Neptuno/git/gen_ai/generative/report.pdf')
print(len(texto_pdf))

Number of PDF pages: 17
51699


In [14]:
print(texto_pdf[0:1000])

February 27, 2025
Norwegian Cruise Line Holdings Reports Strong
Fourth Quarter and Full Year 2024 Financial Results
Strong demand drives 2024 revenue up ~1 1% to full year record
Company beats full year guidance across key metrics and announces 2025 full year guidance
MIAMI, Feb. 27, 2025 (GLOBE NEW SWIRE) -- Norwegian Cruise Line Holdings Ltd. (NYSE: NCLH) (together
with NCL Corporation Ltd. (“NCLC”), “Norwegian Cruise Line Holdings”, “Norwegian”, “NCLH” or the
“Company”) today reported financial results for the fourth quarter and full year ended December 31, 2024 and
provided guidance for the first quarter and full year 2025.
Highlights
Generated 2024 full year record total revenue of $9.5 billion, increasing ~11% over full year 2023 on 3%
capacity growth. GAAP  net income was $910.3 million, up 448% compared to 2023, with EPS increasing
386%, to $1.89.
2024 full year Adjusted EBITDA  grew  32% to a record $2.45 billion, compared to $1.86 billion in 2023 with
Adjusted EPS of $1.82.
T